# Brave Search API Examples

This notebook demonstrates how to use the `BraveSearchClient` for web search functionality.

## Prerequisites

1. Set the `BRAVE_API_KEY` environment variable in your `.env` file
2. Install dependencies via `uv sync`

## Import Required Modules

In [ ]:
import sys
from pathlib import Path

# Add tools_api package to path for imports
# From examples/ we go up to project root, then into tools_api/tools_api
project_root = Path.cwd().parent
tools_api_path = project_root / "tools_api" / "tools_api"
sys.path.insert(0, str(tools_api_path))

from clients.brave_search import (  # noqa: E402
    BraveSearchClient,
    BraveSearchParams,
    SafeSearch,
    Freshness,
    BraveSearchAuthError,
    BraveSearchRateLimitError,
    BraveSearchAPIError,
)

## Example 1: Basic Search

Simple search with a query string.

In [2]:
# Create client (uses BRAVE_API_KEY environment variable)
async with BraveSearchClient() as client:
    # Simple string query
    results = await client.search("Python programming")

    print(f"Query: {results.query.original}")
    print(f"Found {len(results.web.results) if results.web else 0} web results\n")

    # Display first 3 results
    if results.web:
        for i, result in enumerate(results.web.results[:3], 1):
            print(f"{i}. {result.title}")
            print(f"   URL: {result.url}")
            print(f"   Description: {result.description}")
            print()

Query: Python programming
Found 20 web results

1. Welcome to Python.org
   URL: https://www.python.org/
   Description: <strong>Python is a programming language that lets you work quickly and integrate systems more effectively</strong>.

2. Introduction to Python
   URL: https://www.w3schools.com/python/python_intro.asp
   Description: Python has syntax that allows developers to write programs with fewer lines than some other programming languages.

3. Python For Beginners | Python.org
   URL: https://www.python.org/about/gettingstarted/
   Description: The official home of the Python Programming Language



## Example 2: Advanced Search with Parameters

Use `BraveSearchParams` for fine-grained control over search options.

In [3]:
async with BraveSearchClient() as client:
    # Advanced search parameters
    params = BraveSearchParams(
        q="climate change",
        country="US",
        search_lang="en",
        count=5,  # Return 5 results
        safesearch=SafeSearch.MODERATE,
        freshness=Freshness.PAST_WEEK,  # Results from past week
        result_filter="web,news",  # Include web and news results
    )

    results = await client.search(params)

    print(f"Query: {results.query.original}")
    print(f"Country: {results.query.country}")
    print()

    # Web results
    if results.web:
        print(f"Web Results: {len(results.web.results)}")
        for result in results.web.results:
            print(f"  - {result.title}")
            print(f"    {result.url}")
        print()

    # News results
    if results.news:
        print(f"News Results: {len(results.news.results)}")
        for result in results.news.results:
            print(f"  - {result.title}")
            print(f"    {result.url}")
            if result.breaking:
                print("    [BREAKING NEWS]")

Query: climate change
Country: us

Web Results: 5
  - Climate change - Wikipedia
    https://en.wikipedia.org/wiki/Climate_change
  - New reports paint picture of an ‘extremely dangerous’ future with warming expected to blow past key limit | CNN
    https://www.cnn.com/2025/11/12/climate/3-degrees-warming-reports-iea-united-nations
  - What Climate Change Will Do to America by Mid-Century - The Atlantic
    https://www.theatlantic.com/magazine/2025/12/trump-climate-change-acceleration/684632/
  - This chart shows that progress has actually been made on climate change
    https://www.axios.com/2025/11/13/climate-change-progress-chart-co2-emissions
  - World has 'virtually exhausted' its carbon budget as fossil fuel emissions reach all-time high | Euronews
    https://www.euronews.com/green/2025/11/13/world-has-virtually-exhausted-its-carbon-budget-as-fossil-fuel-emissions-reach-all-time-hi

News Results: 7
  - Fight over fossil fuels drawdown looms at UN climate summit
    https://www.f

## Example 3: Location-Based Search

Provide location information for more relevant results.

In [4]:
async with BraveSearchClient() as client:
    # Location information for tailored results
    location = {
        "city": "San Francisco",
        "state": "CA",
        "country": "US",
        "lat": "37.7749",
        "long": "-122.4194",
    }

    results = await client.search(
        "best coffee shops",
        location=location,
    )

    print(f"Query: {results.query.original}")
    print(f"Location Context: {results.query.city}, {results.query.state}")
    print()

    if results.web:
        for i, result in enumerate(results.web.results[:5], 1):
            print(f"{i}. {result.title}")
            print(f"   {result.url}")

Query: best coffee shops
Location Context: San Francisco, CA

1. r/AskSF on Reddit: Recently moved to San Francisco, looking for good coffee place recommendations!
   https://www.reddit.com/r/AskSF/comments/1di0g1l/recently_moved_to_san_francisco_looking_for_good/
2. Home - Top 100 Best Coffee Shops
   https://theworlds100bestcoffeeshops.com/
3. San Francisco’s Best Coffee Shops | Eater SF
   https://sf.eater.com/maps/best-coffee-shops-san-francisco
4. San Francisco | Brian's Coffee Spot
   https://www.brian-coffee-spot.com/the-coffee-spot-guide-to/usa-canada/san-francisco/
5. THE 10 BEST Cafés in San Francisco (Updated 2025) - Tripadvisor
   https://www.tripadvisor.com/Restaurants-g60713-c8-San_Francisco_California.html


## Example 4: Video and News Search

Filter results to specific content types.

In [5]:
async with BraveSearchClient() as client:
    # Search for videos and news only
    params = BraveSearchParams(
        q="machine learning tutorial",
        result_filter="videos,news",
        count=10,
    )

    results = await client.search(params)

    # Video results
    if results.videos:
        print(f"Video Results: {len(results.videos.results)}")
        for video in results.videos.results[:3]:
            print(f"  - {video.title}")
            print(f"    URL: {video.url}")
            if video.duration:
                print(f"    Duration: {video.duration}")
            if video.views:
                print(f"    Views: {video.views:,}")
            print()

    # News results
    if results.news:
        print(f"\nNews Results: {len(results.news.results)}")
        for news in results.news.results[:3]:
            print(f"  - {news.title}")
            print(f"    URL: {news.url}")
            if news.age:
                print(f"    Age: {news.age}")
            print()

Video Results: 5
  - Complete Machine Learning Course in 60 Hours - Part 1 | Full Machine ...
    URL: https://www.youtube.com/watch?v=LcWFedjaR4Q

  - Machine Learning Full Course 2025 | Machine Learning Tutorial For ...
    URL: https://www.youtube.com/watch?v=JnxUsIeTJsM

  - Machine Learning for Everybody – Full Course - YouTube
    URL: https://www.youtube.com/watch?v=i_LwzRVP7bg



## Example 5: Rate Limit Tracking

Monitor API rate limits to avoid exceeding quotas.

In [6]:
async with BraveSearchClient() as client:
    results = await client.search("artificial intelligence")

    # Access rate limit information from the last request
    rate_limit = client.last_rate_limit

    if rate_limit:
        print("Rate Limit Information:")
        print(f"  Limit: {rate_limit.limit}")
        print(f"  Policy: {rate_limit.policy}")
        print(f"  Remaining: {rate_limit.remaining}")
        print(f"  Reset: {rate_limit.reset}s")
        print()

    print(f"Found {len(results.web.results) if results.web else 0} results")

Rate Limit Information:
  Limit: 1, 2000
  Policy: 1;w=1, 2000;w=2592000
  Remaining: 0, 1983
  Reset: 1, 1455523s

Found 20 results


## Example 6: Automatic Rate Limit Handling

**Note**: The client automatically handles the 1 req/sec rate limit by retrying with appropriate delays.

No manual `sleep()` or retry logic needed!

In [7]:
async with BraveSearchClient() as client:
    # Multiple rapid searches - client handles rate limits automatically
    queries = ["Python", "JavaScript", "Rust"]

    print("Making 3 rapid requests (client will auto-retry if rate limited)...\n")

    for query in queries:
        results = await client.search(query)
        print(f"{query}: {len(results.web.results) if results.web else 0} results")

    print("\nRate limit info from last request:")
    if client.last_rate_limit:
        print(f"  Remaining: {client.last_rate_limit.remaining}")

Making 3 rapid requests (client will auto-retry if rate limited)...

Python: 18 results
JavaScript: 20 results
Rust: 20 results

Rate limit info from last request:
  Remaining: 0, 1980


## Example 7: Error Handling

Handle various error conditions gracefully.

In [8]:
async with BraveSearchClient() as client:
    try:
        # This will succeed with valid API key
        results = await client.search("test query")
        print(f"Success! Found {len(results.web.results) if results.web else 0} results")

    except BraveSearchAuthError as e:
        print(f"Authentication Error: {e}")
        print("Check your BRAVE_API_KEY environment variable")

    except BraveSearchRateLimitError as e:
        print(f"Rate Limit Exceeded: {e}")
        print(f"Full info - Remaining: {e.remaining}, Reset: {e.reset}")

    except BraveSearchAPIError as e:
        print(f"API Error: {e}")
        print(f"Status Code: {e.status_code}")

Success! Found 20 results


## Example 8: Custom User Agent and Cache Control

Customize request headers for specific use cases.

In [ ]:
async with BraveSearchClient() as client:
    results = await client.search(
        "current events",
        user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)",
        cache_control="no-cache",  # Bypass cache for fresh results
    )

    print(f"Query: {results.query.original}")
    if results.web:
        print(f"Results: {len(results.web.results)}")
        for result in results.web.results[:3]:
            print(f"  - {result.title}")

## Example 9: Date Range Filtering

Search for content within specific time periods.

In [ ]:
async with BraveSearchClient() as client:
    # Search for recent content (past day)
    params_recent = BraveSearchParams(
        q="breaking news technology",
        freshness=Freshness.PAST_DAY,
        count=5,
    )

    results_recent = await client.search(params_recent)
    print("Results from Past Day:")
    if results_recent.web:
        for result in results_recent.web.results[:3]:
            print(f"  - {result.title}")
            if result.age:
                print(f"    Age: {result.age}")
    print()

    params_year = BraveSearchParams(
        q="climate change research",
        freshness=Freshness.PAST_YEAR,
        count=5,
    )

    results_year = await client.search(params_year)
    print("Results from Past Year:")
    if results_year.web:
        for result in results_year.web.results[:3]:
            print(f"  - {result.title}")

Results from Past Day:
  - USA TODAY - Breaking News and Latest News Today
    Age: 18 minutes ago
  - Tech News - The Latest Technology News | Fox News
    Age: 1 day ago
  - Technology News: Latest Tech News, Phones, Laptops, Gaming & Gadgets Updates | The Indian Express
    Age: 13 hours ago

Results from Past Year:
  - Nature Climate Change
  - Climate change: global temperature | NOAA Climate.gov
  - Recent Research on Climate Change - OEHHA


## Example 10: Safe Search Levels

Control content filtering with different safe search settings.

In [ ]:
async with BraveSearchClient() as client:
    # Strict safe search
    params_strict = BraveSearchParams(
        q="education resources",
        safesearch=SafeSearch.STRICT,
        count=3,
    )

    results = await client.search(params_strict)
    print("Safe Search: STRICT")
    print(f"Family Friendly: {results.web.family_friendly if results.web else 'N/A'}")
    print(f"Results: {len(results.web.results) if results.web else 0}")
    print()

    params_moderate = BraveSearchParams(
        q="education resources",
        safesearch=SafeSearch.MODERATE,
        count=3,
    )

    results = await client.search(params_moderate)
    print("Safe Search: MODERATE")
    print(f"Results: {len(results.web.results) if results.web else 0}")

Safe Search: STRICT
Family Friendly: True
Results: 3

Safe Search: MODERATE
Results: 3


## Example 11: Pagination

Retrieve multiple pages of search results.

In [ ]:
async with BraveSearchClient() as client:
    query = "Python web frameworks"

    # First page
    page1 = BraveSearchParams(q=query, count=10, offset=0)
    results1 = await client.search(page1)

    print("Page 1 Results:")
    if results1.web:
        for i, result in enumerate(results1.web.results[:3], 1):
            print(f"{i}. {result.title}")
    print()

    page2 = BraveSearchParams(q=query, count=10, offset=1)
    results2 = await client.search(page2)

    print("Page 2 Results:")
    if results2.web:
        for i, result in enumerate(results2.web.results[:3], 11):
            print(f"{i}. {result.title}")

    # Check if more results available
    if results2.query.more_results_available:
        print("\nMore results available!")

Page 1 Results:
1. r/Python on Reddit: Which web framework is “king”?
2. 1. Web Frameworks for Python
3. The top 4 Python backend frameworks for building entry level AI projects

Page 2 Results:
11. What non web-oriented Python frameworks exist? - Stack Overflow
12. Python Frameworks 2024: Breaking Down the Best for Your Development Journey | by Prashant Jain | Medium
13. Web Applications & Frameworks — The Hitchhiker's Guide to Python

More results available!


## Example 12: Using with Custom HTTP Client

Pass a custom httpx client for advanced configuration.

In [13]:
async with BraveSearchClient() as client:
    results = await client.search("Python tutorial")

    if results.web:
        for i, result in enumerate(results.web.results[:2], 1):
            print(f"Result {i}:")
            print(f"  Title: {result.title}")
            print(f"  URL: {result.url}")
            print(f"  Description: {result.description}")

            # Meta URL information
            if result.meta_url:
                print(f"  Domain: {result.meta_url.netloc}")
                print(f"  Path: {result.meta_url.path}")
                if result.meta_url.favicon:
                    print(f"  Favicon: {result.meta_url.favicon}")

            # Thumbnail
            if result.thumbnail:
                print(f"  Thumbnail: {result.thumbnail.src}")

            # Additional metadata
            if result.language:
                print(f"  Language: {result.language}")
            if result.age:
                print(f"  Age: {result.age}")

            # Extra snippets
            if result.extra_snippets:
                print(f"  Extra Snippets: {len(result.extra_snippets)} available")

            print()

Result 1:
  Title: The Python Tutorial — Python 3.14.0 documentation
  URL: https://docs.python.org/3/tutorial/index.html
  Description: The Python interpreter is easily ... customizable applications. This tutorial <strong>introduces the reader informally to the basic concepts and features of the Python language and system</strong>....
  Domain: docs.python.org
  Path: › 3  › tutorial  › index.html
  Favicon: https://imgs.search.brave.com/TlYesPBh-Uw2IGKqnRKzWEoKLx3rLOTNmBHkVjoB1ro/rs:fit:32:32:1:0/g:ce/aHR0cDovL2Zhdmlj/b25zLnNlYXJjaC5i/cmF2ZS5jb20vaWNv/bnMvMTUzOTFjOGVi/YTcyOTVmODA3ODIy/YjE2NzFjY2ViMjhl/NzRlY2JhYTc5YjNm/ZjhmODAyZWI2OGUw/ZjU4NDVlNy9kb2Nz/LnB5dGhvbi5vcmcv
  Thumbnail: https://imgs.search.brave.com/SFtCmQlD9qvtG_Qo5ZvS_dut94b__14zYiKL7OhckC8/rs:fit:200:200:1:0/g:ce/aHR0cHM6Ly9kb2Nz/LnB5dGhvbi5vcmcv/My4xNC9faW1hZ2Vz/L3NvY2lhbF9wcmV2/aWV3cy9zdW1tYXJ5/X3R1dG9yaWFsX2lu/ZGV4XzQyMjRlZWY1/LnBuZw
  Language: en

Result 2:
  Title: Python Tutorial
  URL: https://www.w3schools.com/

## Summary

The `BraveSearchClient` provides a comprehensive, type-safe interface for the Brave Search API with:

- **Simple API**: Just `await client.search("query")`
- **Automatic Rate Limiting**: Handles 1 req/sec limit transparently with auto-retry
- **Advanced Parameters**: Fine-grained control with `BraveSearchParams`
- **Error Handling**: Typed exceptions for different error scenarios
- **Rate Limit Tracking**: Monitor API usage via `client.last_rate_limit`
- **Location Support**: Geo-targeted search results
- **Content Filtering**: Safe search, freshness, result types
- **Rich Metadata**: Thumbnails, ratings, timestamps, and more
- **Async Support**: Full async/await compatibility

**Key Feature**: The client automatically retries requests that hit the per-second rate limit, so you never need manual `sleep()` or retry logic in your code!

For production use, integrate with FastAPI using the `get_brave_search_client()` dependency.

## Example 13: Extracting Rich Metadata

Access detailed metadata from search results.

In [14]:
async with BraveSearchClient() as client:
    results = await client.search("Python tutorial")

    if results.web:
        for i, result in enumerate(results.web.results[:2], 1):
            print(f"Result {i}:")
            print(f"  Title: {result.title}")
            print(f"  URL: {result.url}")
            print(f"  Description: {result.description}")

            # Meta URL information
            if result.meta_url:
                print(f"  Domain: {result.meta_url.netloc}")
                print(f"  Path: {result.meta_url.path}")
                if result.meta_url.favicon:
                    print(f"  Favicon: {result.meta_url.favicon}")

            # Thumbnail
            if result.thumbnail:
                print(f"  Thumbnail: {result.thumbnail.src}")

            # Additional metadata
            if result.language:
                print(f"  Language: {result.language}")
            if result.age:
                print(f"  Age: {result.age}")

            # Extra snippets
            if result.extra_snippets:
                print(f"  Extra Snippets: {len(result.extra_snippets)} available")

            print()

Result 1:
  Title: The Python Tutorial — Python 3.14.0 documentation
  URL: https://docs.python.org/3/tutorial/index.html
  Description: The Python interpreter is easily ... customizable applications. This tutorial <strong>introduces the reader informally to the basic concepts and features of the Python language and system</strong>....
  Domain: docs.python.org
  Path: › 3  › tutorial  › index.html
  Favicon: https://imgs.search.brave.com/TlYesPBh-Uw2IGKqnRKzWEoKLx3rLOTNmBHkVjoB1ro/rs:fit:32:32:1:0/g:ce/aHR0cDovL2Zhdmlj/b25zLnNlYXJjaC5i/cmF2ZS5jb20vaWNv/bnMvMTUzOTFjOGVi/YTcyOTVmODA3ODIy/YjE2NzFjY2ViMjhl/NzRlY2JhYTc5YjNm/ZjhmODAyZWI2OGUw/ZjU4NDVlNy9kb2Nz/LnB5dGhvbi5vcmcv
  Thumbnail: https://imgs.search.brave.com/SFtCmQlD9qvtG_Qo5ZvS_dut94b__14zYiKL7OhckC8/rs:fit:200:200:1:0/g:ce/aHR0cHM6Ly9kb2Nz/LnB5dGhvbi5vcmcv/My4xNC9faW1hZ2Vz/L3NvY2lhbF9wcmV2/aWV3cy9zdW1tYXJ5/X3R1dG9yaWFsX2lu/ZGV4XzQyMjRlZWY1/LnBuZw
  Language: en

Result 2:
  Title: Python Tutorial
  URL: https://www.w3schools.com/

---